# Blister Detection using YOLOv8

This notebook implements blister detection using YOLOv8 with a two-stage training approach:
1. Train on synthetic data to learn general blister features
2. Fine-tune on real blister images for improved real-world performance

## Our Approach & Results
- Base Model: YOLOv8s
- First trained on synthetic data achieving:
  - Precision: 97.5%
  - Recall: 85.3%
  - mAP50: 90.5%
  - mAP50-95: 78.1%
- Then fine-tuned on real blister images

In [ ]:
# Install required packages
!pip install ultralytics
!pip install torch torchvision

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Disable wandb notifications
import os
os.environ['WANDB_MODE'] = 'disabled'

## Dataset Structure

### 1. Synthetic Dataset
Located at: `/content/drive/MyDrive/unelmat_ai/synthetic_dataset/`
- Purpose: Initial training to learn basic blister features
- Contains: Large number of synthetic blister images
- Statistics: 500 validation images with 20,004 blister instances

### 2. Real Dataset
Located at: `/content/drive/MyDrive/unelmat_ai/yolo_dataset`
- Purpose: Fine-tuning for real-world performance
- Contains: Real blister images for domain adaptation

Both datasets follow YOLO format.


In [ ]:
# # Create YAML configuration for synthetic dataset
# %%writefile synthetic_dataset.yaml
# path: /content/drive/MyDrive/unelmat_ai/synthetic_dataset
# train: images/train
# val: images/val
# test: images/test
# nc: 2
# names: ['blister', 'defect']

# print("Synthetic dataset configuration created!")

## Training Strategy

### Stage 1: Synthetic Data Training
- Purpose: Learn basic blister features
- Model: YOLOv8s
- Key Parameters:
  - Learning Rate: 0.001
  - Batch Size: 16
  - Image Size: 640x640
  - Early Stopping Patience: 50

### Stage 2: Real Data Fine-tuning
- Purpose: Adapt to real-world conditions
- Base: Best weights from synthetic training
- Key Parameters:
  - Lower Learning Rate: 0.0005
  - Same architecture and image size
  - Fine-tuning specific adjustments

### Stage 3. Combined Augmented Dataset
Located at: `/content/drive/MyDrive/unelmat_ai/combined_dataset`
- Purpose: Enhanced training with diverse data
- Contains: Combination of
 * Original real blister images
 * Synthetic blister images
 * Augmented variations:
   - Flipped images
   - Rotated versions (±15 degrees)
   - Brightness adjusted
   - Contrast variations
   - Blur variations
   - Images with added noise
- Benefits:
 * Increased dataset size
 * Better model generalization
 * Improved robustness
 * Enhanced performance on edge cases

In [ ]:
from ultralytics import YOLO
import torch

# Clear GPU cache
torch.cuda.empty_cache()

# Load YOLOv8s model
model = YOLO('/content/drive/MyDrive/unelmat_ai/yolov8s.pt')

# Train on synthetic data
results = model.train(
    data='/content/drive/MyDrive/unelmat_ai/synthetic_dataset/synthetic_dataset.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    save_dir='/content/drive/MyDrive/unelmat_ai/yolo_results',
    name='yolov8s_synthetic',
    exist_ok=True,
    device=0,
    workers=8,
    patience=50,
    lr0=0.001,
    lrf=0.0001,
    warmup_epochs=3,
    mosaic=0.0,
    cache=True
)

In [ ]:
# Fine Tuning
# Load the synthetic-trained model
model = YOLO('/content/drive/MyDrive/unelmat_ai/yolo_results/yolov8s_synthetic/weights/best.pt')

# Fine-tune on real data
results = model.train(
    data='/content/yolo_dataset/yolo_dataset/yolo_dataset.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    save_dir='/content/drive/MyDrive/unelmat_ai/yolo_results',
    name='yolov8s_real_finetuned',
    exist_ok=True,
    device=0,
    workers=8,
    patience=50,
    lr0=0.0005,  # Lower learning rate for fine-tuning
    lrf=0.0001,
    warmup_epochs=1,
    mosaic=0.0,
    cache=True
)

# Testing Different Model Weights with Gradio Interface

## Available Model Weights

We have trained several models using different approaches. You can experiment with any of these weights:

### 1. Combined Dataset Model
- Location: `/content/drive/MyDrive/unelmat_ai/yolo_results/yolov8s_combined/weights/best.pt`
- Trained on: Synthetic + Real + Augmented data
- Best for: Overall robust performance
- Use when: Need balanced performance on various blister types

### 2. Synthetic Data Model
- Location: `/content/drive/MyDrive/unelmat_ai/yolo_results/yolov8s_synthetic/weights/best.pt`
- Trained on: Synthetic blister images
- Performance:
 * Precision: 97.5%
 * Recall: 85.3%
 * mAP50: 90.5%
- Use when: Testing on synthetic-like images

### 3. Fine-tuned Model
- Location: `/content/drive/MyDrive/unelmat_ai/yolo_results/yolov8s_real_finetuned/weights/best.pt`
- Trained on: Real data after synthetic training
- Use when: Testing on real-world images

### 4. Previous Experiment Models
Location: `/content/drive/MyDrive/unelmat_ai/runs/detect/`

## Interactive Testing Interface
Below is a Gradio interface where you can:
- Select different model weights
- Adjust confidence threshold
- Test on your own images
- Compare results between models

In [ ]:
# Install Gradio
!pip install gradio

# Import required libraries
import gradio as gr
from ultralytics import YOLO
import cv2
import numpy as np
from PIL import Image
import os

In [12]:
# Load model and interface
# model = YOLO('/content/drive/MyDrive/unelmat_ai/runs/detect/yolov8s_real_finetuned/weights/best.pt')

# Alternative paths to try:
# model = YOLO('/content/drive/MyDrive/unelmat_ai/yolo_results/yolov8s_combined/weights/best.pt')
# or
model = YOLO('/content/drive/MyDrive/unelmat_ai/runs/detect/yolov8s_synthetic/weights/best.pt')

# HERE, blister grading is not according to instruction, its just a simple way to understand how are the detected blisters.
def grade_blisters(num_blisters):
    """Grade based on number of blisters"""
    if num_blisters == 0:
        return "Grade A - No Blisters"
    elif num_blisters <= 2:
        return "Grade B - Minor Blisters"
    elif num_blisters <= 5:
        return "Grade C - Moderate Blisters"
    else:
        return "Grade D - Severe Blisters"

def analyze_blisters(images):
    """Analyze multiple images for blisters"""
    results_all = []

    for idx, image in enumerate(images):
        # Perform prediction
        results = model.predict(image, conf=0.25)
        result_image = results[0].plot()
        result_image = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)

        # Get detection details
        boxes = results[0].boxes
        num_blisters = len(boxes)

        # Calculate confidence scores
        confidences = [float(box.conf[0]) for box in boxes]
        avg_confidence = np.mean(confidences) if confidences else 0

        # Analyze blister sizes
        sizes = []
        if len(boxes.xyxy) > 0:
            for box in boxes.xyxy:
                width = float(box[2] - box[0])
                height = float(box[3] - box[1])
                area = width * height
                sizes.append(area)
        avg_size = np.mean(sizes) if sizes else 0

        # Grade the image
        grade = grade_blisters(num_blisters)

        # Create analysis summary
        summary = f"Image {idx + 1} Analysis:\n"
        summary += f"Number of Blisters: {num_blisters}\n"
        summary += f"Average Confidence: {avg_confidence:.2%}\n"
        summary += f"Average Blister Size: {avg_size:.2f} pixels²\n"
        summary += f"Grade: {grade}\n"

        # Categorize blisters by confidence
        high_conf = len([c for c in confidences if c > 0.8])
        med_conf = len([c for c in confidences if 0.5 < c <= 0.8])
        low_conf = len([c for c in confidences if c <= 0.5])

        summary += "\nBlister Categories:\n"
        summary += f"High Confidence (>80%): {high_conf}\n"
        summary += f"Medium Confidence (50-80%): {med_conf}\n"
        summary += f"Low Confidence (<50%): {low_conf}\n"

        # Size classification
        if sizes:
            small = len([s for s in sizes if s < np.percentile(sizes, 33)])
            medium = len([s for s in sizes if np.percentile(sizes, 33) <= s < np.percentile(sizes, 66)])
            large = len([s for s in sizes if s >= np.percentile(sizes, 66)])

            summary += "\nSize Distribution:\n"
            summary += f"Small Blisters: {small}\n"
            summary += f"Medium Blisters: {medium}\n"
            summary += f"Large Blisters: {large}\n"

        results_all.append((result_image, summary))

    # Prepare outputs
    images_out = [res[0] for res in results_all]
    summaries = [res[1] for res in results_all]

    combined_summary = "\n\n".join(summaries)

    return images_out, combined_summary

# Create enhanced Gradio interface
interface = gr.Interface(
    fn=analyze_blisters,
    inputs=gr.File(
        file_count="multiple",
        file_types=["image"],
        label="Upload Multiple Images"
    ),
    outputs=[
        gr.Gallery(
            label="Detection Results",
            show_label=True,
            columns=2,
            height="auto"
        ),
        gr.Textbox(
            label="Analysis Summary",
            lines=10
        )
    ],
    title="Advanced Blister Detection and Analysis",
    description="""
    Upload one or more images to:
    1. Detect blisters in each image
    2. Get count of blisters
    3. See confidence levels
    4. Get size analysis
    5. Receive quality grade
    6. View categorization by size and confidence
    """,
    examples=[
        # Add example images if available
        # ["example1.jpg"],
        # ["example2.jpg"]
    ],
    theme="default"
)

# Custom CSS for better layout
interface.launch(
    share=True,

)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://49f1ec8883845ce3c8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
